# V / Modélisation avec l'algorithme Autoencoder


## V.1 ) Classification Binaire


In [ ]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.ndimage import zoom
from tensorflow.keras.callbacks import EarlyStopping



# Function to load and preprocess NIfTI images
def load_and_preprocess_images_nifti_batch(image_paths, target_size):
    images = []
    for image_path in image_paths:
        img = nib.load(image_path).get_fdata()
        img_resized = zoom(img, target_size / np.array(img.shape), order=1)
        images.append(img_resized)
    return np.array(images)

# Set directories and file paths
image_dir = 'E:/StageEsprit/ADNI_CNN_Transfer_Autoencoder/annual_screening/annual/ADNI/CN,AD,MCI'
excel_path = 'E:/StageEsprit/ADNI_CNN_Transfer_Autoencoder/ADNI1_Annual_2_Yr_3T_8_05_2023 - Copie.xlsx'

# Load labels from Excel file
df = pd.read_excel(excel_path)

# List of allowed image extensions
image_extensions = ['.nii', '.nii.gz']

# List to store image paths
image_paths = []

# Collect image paths from the directory
for file_name in os.listdir(image_dir):
    if any(file_name.lower().endswith(ext) for ext in image_extensions):
        image_path = os.path.join(image_dir, file_name)
        image_paths.append(image_path)

# Target size for image resizing
target_size_reduced = (224, 224, 3)

# Batch size for loading images
batch_size = 5

# Split image paths into batches
image_batches = [image_paths[i:i + batch_size] for i in range(0, len(image_paths), batch_size)]

# Load and preprocess images in batches
images_nifti = []
y_labels = []

for batch_paths in image_batches:
    batch_images = load_and_preprocess_images_nifti_batch(batch_paths, target_size_reduced)
    images_nifti.append(batch_images)

    batch_labels = []
    for image_path in batch_paths:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        matching_rows = df[df['Subject'].str.contains(image_name)]['Group']
        if len(matching_rows) > 0:
            label = matching_rows.values[0]
            if label == 'CN':
                batch_labels.append(0)
            else:
                batch_labels.append(1)
            

    y_labels.extend(batch_labels)

images_nifti = np.concatenate(images_nifti, axis=0)
y_labels = np.array(y_labels)


# Split data into train, test, and validation sets (70% train, 30% test)
train_images, test_images, y_train, y_test = train_test_split(images_nifti, y_labels, test_size=0.3, random_state=42)
test_images, val_images, y_test, y_val = train_test_split(test_images, y_test, test_size=0.5, random_state=42)




# Encoder part of the autoencoder
input_shape = (224, 224, 3)

autoencoder = tf.keras.Sequential()
autoencoder.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))

# Decoder part of the autoencoder
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))



# Compile autoencoder with adjusted learning rate
#autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss='mse', metrics=['accuracy'])
autoencoder.compile(loss='mse', metrics=['accuracy'])


# Normalize the pixel values to [0, 1] range
train_images_normalized = train_images / 255.0
test_images_normalized = test_images / 255.0
val_images_normalized = val_images / 255.0

# Train the autoencoder on the normalized training images with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = autoencoder.fit(train_images_normalized, train_images_normalized, batch_size=32, epochs=10, shuffle=True, validation_data=(val_images_normalized, val_images_normalized), callbacks=[early_stopping])





# Extract encoded representations of the training and test images
encoded_train_images = autoencoder.predict(train_images)
encoded_test_images = autoencoder.predict(test_images)



from sklearn.svm import SVC

# Reshape the encoded images to 2D arrays
encoded_train_images_flattened = encoded_train_images.reshape(encoded_train_images.shape[0], -1)
encoded_test_images_flattened = encoded_test_images.reshape(encoded_test_images.shape[0], -1)

# Initialize SVM classifier
svm_classifier = SVC(kernel='rbf', C=10, class_weight='balanced')

# Train the SVM classifier on the flattened encoded representations
svm_classifier.fit(encoded_train_images_flattened, y_train)

# Predict using the SVM classifier
svm_predictions = svm_classifier.predict(encoded_test_images_flattened)

# Calculate evaluation metrics for SVM classifier
accuracy = accuracy_score(y_test, svm_predictions)
precision = precision_score(y_test, svm_predictions, average='weighted',zero_division = 1)
recall = recall_score(y_test, svm_predictions, average='weighted',zero_division = 1)
f1 = f1_score(y_test, svm_predictions, average='weighted',zero_division = 1)
conf_matrix = confusion_matrix(y_test, svm_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

## Affichage des images de test en classification binaire 

In [ ]:
import math

# Nouvelles dimensions pour l'affichage des images
display_width = 8
display_height = 10

# Calcul du nombre de colonnes en fonction du nombre d'images et des dimensions d'affichage
num_cols = 8
num_rows = math.ceil(len(test_images) / num_cols)

# Création de la grille pour afficher les images
fig, axes = plt.subplots(num_rows, num_cols, figsize=(display_width * num_cols, display_height * num_rows))

# Mapping des noms de classes aux étiquettes
class_names = {0: 'Non Alzheimer (CN)', 1: 'Alzheimer'}

for i, ax in enumerate(axes.flat):
    if i < len(test_images):
        img = convert_to_grayscale(test_images[i])
        predicted_label = svm_predictions[i]
        
        ax.imshow(img, cmap='gray')  # Display in grayscale
        
        predicted_class = class_names[predicted_label]
        
        ax.set_title(f"Prédiction: {predicted_class}", fontsize=35)
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()


## Affichage des images originales et de reconstruction en classification binaire 

In [ ]:
# Function to convert RGB images to grayscale
def convert_to_grayscale(images):
    return np.dot(images[...,:3], [0.2989, 0.5870, 0.1140])

# Get the reconstructed images from all test images
reconstructed_images = autoencoder.predict(test_images_normalized)

# Set the number of rows and columns for the grid
num_rows = 2
num_columns = 3
num_images_to_display = num_rows * num_columns

# Create a figure and subplots
plt.figure(figsize=(15, 10))

for i in range(num_images_to_display):
    # Original image
    plt.subplot(num_rows, 2 * num_columns, 2 * i + 1)
    plt.imshow(convert_to_grayscale(test_images[i]), cmap='gray')
    plt.title("Original", fontsize=10)
    plt.axis("off")
    
    # Reconstructed image
    plt.subplot(num_rows, 2 * num_columns, 2 * i + 2)
    plt.imshow(convert_to_grayscale(reconstructed_images[i]), cmap='gray')
    plt.title("Reconstructed", fontsize=10)
    plt.axis("off")

plt.tight_layout()
plt.show()


## Calcul de l'erreur de reconstruction (mse) en classification binaire

In [ ]:
# Calculate the mean squared error (MSE) between original and reconstructed images
mse = np.mean(np.square(test_images_normalized - reconstructed_images))
print(mse)

## V.2) Classification Multiclasse


In [ ]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from scipy.ndimage import zoom
from tensorflow.keras.callbacks import EarlyStopping



# Function to load and preprocess NIfTI images
def load_and_preprocess_images_nifti_batch(image_paths, target_size):
    images = []
    for image_path in image_paths:
        img = nib.load(image_path).get_fdata()
        img_resized = zoom(img, target_size / np.array(img.shape), order=1)
        images.append(img_resized)
    return np.array(images)

# Set directories and file paths
image_dir = 'E:/StageEsprit/ADNI_CNN_Transfer_Autoencoder/annual_screening/annual/ADNI/CN,AD,MCI'
excel_path = 'E:/StageEsprit/ADNI_CNN_Transfer_Autoencoder/ADNI1_Annual_2_Yr_3T_8_05_2023 - Copie.xlsx'

# Load labels from Excel file
df = pd.read_excel(excel_path)

# List of allowed image extensions
image_extensions = ['.nii', '.nii.gz']

# List to store image paths
image_paths = []

# Collect image paths from the directory
for file_name in os.listdir(image_dir):
    if any(file_name.lower().endswith(ext) for ext in image_extensions):
        image_path = os.path.join(image_dir, file_name)
        image_paths.append(image_path)

# Target size for image resizing
target_size_reduced = (224, 224, 3)

# Batch size for loading images
batch_size = 5

# Split image paths into batches
image_batches = [image_paths[i:i + batch_size] for i in range(0, len(image_paths), batch_size)]

# Load and preprocess images in batches
images_nifti = []
y_labels = []

for batch_paths in image_batches:
    batch_images = load_and_preprocess_images_nifti_batch(batch_paths, target_size_reduced)
    images_nifti.append(batch_images)

    batch_labels = []
    for image_path in batch_paths:
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        matching_rows = df[df['Subject'].str.contains(image_name)]['Group']
        if len(matching_rows) > 0:
            label = matching_rows.values[0]
            if label == 'CN':
                batch_labels.append(0)
            elif label == 'AD':
                batch_labels.append(1)
            elif label == 'MCI':
                batch_labels.append(2)

    y_labels.extend(batch_labels)

images_nifti = np.concatenate(images_nifti, axis=0)
y_labels = np.array(y_labels)


# Split data into train, test, and validation sets (60% train, 20% test, 20% validation)
train_images, test_images, y_train, y_test = train_test_split(images_nifti, y_labels, test_size=0.4, random_state=42)
test_images, val_images, y_test, y_val = train_test_split(test_images, y_test, test_size=0.5, random_state=42)



# Encoder part of the autoencoder
input_shape = (224, 224, 3)

autoencoder = tf.keras.Sequential()
autoencoder.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(MaxPooling2D((2, 2), padding='same'))

# Decoder part of the autoencoder
autoencoder.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(Conv2D(3, (3, 3), activation='softmax', padding='same'))



# Compile autoencoder with adjusted learning rate
autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), 
                    loss='categorical_crossentropy', metrics=['accuracy'])


# Normalize the pixel values to [0, 1] range
train_images_normalized = train_images / 255.0
test_images_normalized = test_images / 255.0
val_images_normalized = val_images / 255.0

# Train the autoencoder on the normalized training images with early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = autoencoder.fit(train_images_normalized, train_images_normalized, batch_size=64, epochs=20, shuffle=True, 
                          validation_data=(val_images_normalized, val_images_normalized), callbacks=[early_stopping])





# Extract encoded representations of the training and test images
encoded_train_images = autoencoder.predict(train_images)
encoded_test_images = autoencoder.predict(test_images)



from sklearn.svm import SVC

# Reshape the encoded images to 2D arrays
encoded_train_images_flattened = encoded_train_images.reshape(encoded_train_images.shape[0], -1)
encoded_test_images_flattened = encoded_test_images.reshape(encoded_test_images.shape[0], -1)

# Initialize SVM classifier
svm_classifier = SVC(kernel='rbf', C=10, class_weight='balanced')

# Train the SVM classifier on the flattened encoded representations
svm_classifier.fit(encoded_train_images_flattened, y_train)

# Predict using the SVM classifier
svm_predictions = svm_classifier.predict(encoded_test_images_flattened)

# Calculate evaluation metrics for SVM classifier
accuracy = accuracy_score(y_test, svm_predictions)
precision = precision_score(y_test, svm_predictions, average='weighted')
recall = recall_score(y_test, svm_predictions, average='weighted')
f1 = f1_score(y_test, svm_predictions, average='weighted')
conf_matrix = confusion_matrix(y_test, svm_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

## Affichage des images de test en classification multiclasse 

In [ ]:
import math

# Nouvelles dimensions pour l'affichage des images
display_width = 8
display_height = 10

# Calcul du nombre de colonnes en fonction du nombre d'images et des dimensions d'affichage
num_cols = 8
num_rows = math.ceil(len(test_images) / num_cols)

# Création de la grille pour afficher les images
fig, axes = plt.subplots(num_rows, num_cols, figsize=(display_width * num_cols, display_height * num_rows))

# Mapping des noms de classes aux étiquettes
class_names = {0: 'Non Alzheimer (CN)', 1: 'Alzheimer (AD)', 2: 'Alzheimer (MCI)'}

for i, ax in enumerate(axes.flat):
    if i < len(test_images):
        img = convert_to_grayscale(test_images[i])
        predicted_label = svm_predictions[i]
        
        ax.imshow(img, cmap='gray')  # Display in grayscale
        
        predicted_class = class_names[predicted_label]
        
        ax.set_title(f"Prédiction: {predicted_class}", fontsize=35)
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()


## Affichage des images originales et de reconstruction en classification multiclasse 

In [ ]:
# Function to convert RGB images to grayscale
def convert_to_grayscale(images):
    return np.dot(images[...,:3], [0.2989, 0.5870, 0.1140])

# Get the reconstructed images from all test images
reconstructed_images = autoencoder.predict(test_images_normalized)

# Set the number of rows and columns for the grid
num_rows = 2
num_columns = 4
num_images_to_display = num_rows * num_columns

# Randomly choose indices for images to display
random_indices = np.random.choice(len(test_images), num_images_to_display, replace=False)

# Create a figure and subplots
plt.figure(figsize=(15, 5))

for i, idx in enumerate(random_indices):
    # Original image
    plt.subplot(num_rows, 2 * num_columns, 2 * i + 1)
    plt.imshow(convert_to_grayscale(test_images[idx]), cmap='gray')
    plt.title("Original", fontsize=10)
    plt.axis("off")
    
    # Reconstructed image
    plt.subplot(num_rows, 2 * num_columns, 2 * i + 2)
    plt.imshow(convert_to_grayscale(reconstructed_images[idx]), cmap='gray')
    plt.title("Reconstructed", fontsize=10)
    plt.axis("off")

plt.tight_layout()
plt.show()


## Calcul de l'erreur de reconstruction (mse) en classification multiclasse

In [ ]:
# Calculate the mean squared error (MSE) between original and reconstructed images
mse = np.mean(np.square(test_images_normalized - reconstructed_images))
print(mse)

# VI.1) Comparaison entre les modèles pour la classificatoin binaire

In [ ]:
# Définir les métriques pour chaque algorithme
accuracy_cnn = 0.7973856209150327  # Remplacez par la valeur de l'accuracy pour CNN
precision_cnn = 0.8354430379746836 # Remplacez par la valeur de la precision pour CNN
recall_cnn = 0.7857142857142857   # Remplacez par la valeur du recall pour CNN
f1_cnn = 0.8098159509202455  # Remplacez par la valeur du F1-score pour CNN


accuracy_transfer = 0.8608695652173913  # Remplacez par la valeur de l'accuracy pour Transfer Learning
precision_transfer = 0.8947368421052632  # Remplacez par la valeur de la precision pour Transfer Learning
recall_transfer = 0.8360655737704918  # Remplacez par la valeur du recall pour Transfer Learning
f1_transfer = 0.864406779661017   # Remplacez par la valeur du F1-score pour Transfer Learning

accuracy_autoencoder = 0.7543859649122807  # Remplacez par la valeur de l'accuracy pour Autoencoder
precision_autoencoder = 0.779337231968811   # Remplacez par la valeur de la precision pour Autoencoder
recall_autoencoder = 0.7543859649122807  # Remplacez par la valeur du recall pour Autoencoder
f1_autoencoder = 0.758572567783094  # Remplacez par la valeur du F1-score pour Autoencoder


# Créer le graphe à barres avec les métriques définies ci-dessus
import matplotlib.pyplot as plt
import numpy as np

# Métriques pour les trois algorithmes
algorithms = ['CNN', 'Transfer Learning', 'Autoencoder']
accuracy_scores = [accuracy_cnn, accuracy_transfer, accuracy_autoencoder]
precision_scores = [precision_cnn, precision_transfer, precision_autoencoder]
recall_scores = [recall_cnn, recall_transfer, recall_autoencoder]
f1_scores = [f1_cnn, f1_transfer, f1_autoencoder]

# Créer une figure avec plusieurs sous-graphiques
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

# Métrique : Accuracy
axes[0, 0].bar(algorithms, accuracy_scores, color='blue')
axes[0, 0].set_title('Accuracy')
axes[0, 0].set_ylim(0, 1)

# Métrique : Precision
axes[0, 1].bar(algorithms, precision_scores, color='green')
axes[0, 1].set_title('Precision')
axes[0, 1].set_ylim(0, 1)

# Métrique : Recall
axes[1, 0].bar(algorithms, recall_scores, color='orange')
axes[1, 0].set_title('Recall')
axes[1, 0].set_ylim(0, 1)

# Métrique : F1-Score
axes[1, 1].bar(algorithms, f1_scores, color='red')
axes[1, 1].set_title('F1-Score')
axes[1, 1].set_ylim(0, 1)

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher le graphe
plt.show()


# VI.2) Comparaison entre les modèles pour la classificatoin multiclasse

In [ ]:
# Définir les métriques pour chaque algorithme
accuracy_cnn = 0.7189542483660131  # Remplacez par la valeur de l'accuracy pour CNN
precision_cnn = 0.6790242165242165 # Remplacez par la valeur de la precision pour CNN
recall_cnn = 0.6504667998356638   # Remplacez par la valeur du recall pour CNN
f1_cnn = 0.6604542573275766  # Remplacez par la valeur du F1-score pour CNN

accuracy_transfer = 0.8173913043478261  # Remplacez par la valeur de l'accuracy pour Transfer Learning
precision_transfer = 0.8157886634967331  # Remplacez par la valeur de la precision pour Transfer Learning
recall_transfer = 0.8173913043478261  # Remplacez par la valeur du recall pour Transfer Learning
f1_transfer = 0.8124937531234382   # Remplacez par la valeur du F1-score pour Transfer Learning

accuracy_autoencoder = 0.6842105263157895  # Remplacez par la valeur de l'accuracy pour Autoencoder
precision_autoencoder = 0.687427997761759   # Remplacez par la valeur de la precision pour Autoencoder
recall_autoencoder = 0.6842105263157895  # Remplacez par la valeur du recall pour Autoencoder
f1_autoencoder = 0.6782862205299879  # Remplacez par la valeur du F1-score pour Autoencoder


# Créer le graphe à barres avec les métriques définies ci-dessus
import matplotlib.pyplot as plt
import numpy as np

# Métriques pour les trois algorithmes
algorithms = ['CNN', 'Transfer Learning', 'Autoencoder']
accuracy_scores = [accuracy_cnn, accuracy_transfer, accuracy_autoencoder]
precision_scores = [precision_cnn, precision_transfer, precision_autoencoder]
recall_scores = [recall_cnn, recall_transfer, recall_autoencoder]
f1_scores = [f1_cnn, f1_transfer, f1_autoencoder]

# Créer une figure avec plusieurs sous-graphiques
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

# Métrique : Accuracy
axes[0, 0].bar(algorithms, accuracy_scores, color='blue')
axes[0, 0].set_title('Accuracy')
axes[0, 0].set_ylim(0, 1)

# Métrique : Precision
axes[0, 1].bar(algorithms, precision_scores, color='green')
axes[0, 1].set_title('Precision')
axes[0, 1].set_ylim(0, 1)

# Métrique : Recall
axes[1, 0].bar(algorithms, recall_scores, color='orange')
axes[1, 0].set_title('Recall')
axes[1, 0].set_ylim(0, 1)

# Métrique : F1-Score
axes[1, 1].bar(algorithms, f1_scores, color='red')
axes[1, 1].set_title('F1-Score')
axes[1, 1].set_ylim(0, 1)

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher le graphe
plt.show()


# VII.1) Interprétation des résultats pour la classificatoin binaire 

Les trois modèles, à savoir le CNN, le Transfer Learning et l'Autoencoder, ont été évalués en termes 
de leurs performances dans une tâche de classification binaire. En comparant leurs métriques clés, 
nous pouvons observer ce qui suit :


CNN : Le modèle Convolutional Neural Network (CNN) a obtenu une accuracy de 79.74%, ce qui indique 
que près de 80% des prédictions étaient correctes. La précision de 83.54% souligne la capacité du modèle 
à prédire précisément les exemples positifs. Le recall élevé de 78.57% signifie que le modèle a réussi 
à identifier la majorité des vrais positifs parmi les exemples positifs réels. Le F1-score de 80.98% combine 
ces métriques pour donner une vue équilibrée de la performance du modèle.


Transfer Learning : Le modèle de Transfer Learning a surpassé les autres modèles en termes d'accuracy avec 86.09%, 
indiquant que c'est celui qui a généré le plus grand nombre de prédictions correctes globalement. Avec une précision 
de 89.47%, il a montré une forte capacité à prédire correctement les exemples positifs. Son recall de 83.61% 
et son F1-score de 86.44% montrent qu'il a réussi à trouver un équilibre entre la détection des vrais positifs 
et la minimisation des faux négatifs.


Autoencoder : L'Autoencoder a également obtenu des résultats compétitifs avec une accuracy de 75.44%. 
Sa précision de 77.93% indique une bonne capacité à prédire les exemples positifs, tandis que son recall de 75.44% 
montre qu'il a identifié la majorité des vrais positifs. Le F1-score de 75.86% reflète un compromis solide 
entre la précision et le recall.

En conclusion, bien que chaque modèle ait ses propres points forts, le modèle de Transfer Learning 
s'est démarqué en termes d'accuracy globale, tandis que le CNN et l'Autoencoder ont montré des performances 
équilibrées en termes de précision et de recall. Chacun de ces modèles pourrait être adapté en fonction des priorités
spécifiques de la tâche et du compromis souhaité entre la précision et la détection.

# VII.2) Interprétation des résultats pour la classificatoin multiclasse 

Les trois modèles, à savoir le CNN, le Transfer Learning et l'Autoencoder, ont été évalués en termes de leurs performances dans une tâche de classification multiclasse. En comparant leurs métriques clés, nous pouvons observer ce qui suit :



CNN : Le modèle Convolutional Neural Network (CNN) a obtenu une accuracy de 71.90%, ce qui indique que près de 72% des prédictions étaient correctes. La précision de 67.90% souligne la capacité du modèle à prédire précisément les exemples positifs. Le recall de 65.05% signifie que le modèle a réussi à identifier plus de la moitié des vrais positifs parmi les exemples positifs réels. Le F1-score de 66.05% combine ces métriques pour donner une vue équilibrée de la performance du modèle.



Transfer Learning : Le modèle de Transfer Learning a surpassé les autres modèles en termes d'accuracy avec 81.74%, 
indiquant que c'est celui qui a généré le plus grand nombre de prédictions correctes globalement. Avec une précision de 81.58%, il a montré une forte capacité à prédire correctement les exemples positifs. Son recall de 81.74% et son F1-score de 81.25% montrent qu'il a réussi à trouver un équilibre entre la détection des vrais positifs et la minimisation des faux négatifs.



Autoencoder : L'Autoencoder a également obtenu des résultats compétitifs avec une accuracy de 68.42%. Sa précision de 68.74% indique une bonne capacité à prédire les exemples positifs, tandis que son recall de 68.42% montre qu'il a identifié la majorité des vrais positifs. Le F1-score de 67.83% reflète un compromis solide entre la précision et le recall.


En conclusion, bien que chaque modèle ait ses propres points forts, le modèle de Transfer Learning s'est démarqué en termes d'accuracy globale, tandis que le CNN et l'Autoencoder ont montré des performances équilibrées en termes de précision et de recall. Chacun de ces modèles pourrait être adapté en fonction des priorités spécifiques de la tâche et du compromis souhaité entre la précision et la détection.

###  VIII ) Comparaison des Performances de l'Autoencoder en Fonction de Deux Approches de Classification en utilisant la Métrique de l'Erreur de Reconstruction (MSE)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Reconstruction errors for binary classification and multiclass classification
reconstruction_error_binary = 0.346050837213107   # Replace with the reconstruction error for binary classification
reconstruction_error_multiclass = 0.278545210648486  # Replace with the reconstruction error for multiclass classification

# Create a figure
plt.figure(figsize=(6, 4))

# Plot reconstruction errors
plt.bar(['Binary Classification', 'Multiclass Classification'], [reconstruction_error_binary, reconstruction_error_multiclass], color=['blue', 'green'])

# Add labels and title
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error Comparison')

# Show the plot
plt.show()


### IX) Interprétation de Comparaison des Performances de l'Autoencoder en Fonction de Deux Approches de Classification en Utilisant la Métrique de l'Erreur de Reconstruction (MSE)

In [ ]:
La comparaison entre la classification binaire et multiclasse au niveau de la métrique 
d'erreur de reconstruction (MSE) révèle que la méthode de classification multiclasse présente
une erreur de reconstruction moyenne (MSE) plus faible (0.278545210648486) par rapport 
à la classification binaire (0.346050837213107). Cela indique que le modèle de classification multiclasse 
est capable de mieux reconstruire les images d'origine à partir de leurs caractéristiques latentes
que le modèle de classification binaire. En d'autres termes, le modèle multiclasse a une meilleure capacité 
de représentation et de reconstitution des données par rapport au modèle binaire
en utilisant l'approche d'autoencodeur.